In [1]:
from importlib import reload
import os
import pandas as pd
import numpy as np
import random as rd

from pathlib import Path
from sqlalchemy.orm import sessionmaker
import matplotlib.pyplot as plt
from pprint import pprint
import seaborn as sns
import statsmodels.api as sma
import statsmodels.stats.outliers_influence  as smo
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import joblib

import sqlite_io as sio
import add_indicators as indic
import split_merge as sm
import balance
import model_mngr as modmgr

reload(modmgr)

<module 'model_mngr' from 'G:\\Python\\MarketDataEnrichment\\dataset_mngr\\model_mngr.py'>

In [2]:
# PATH_DATA = "C:\\Projets\\Data"
PATH_DATA = "G:\\Python\\Data"
PATH_DB_FWK=PATH_DATA+"\\sqlite\\dataset_market.db"
PATH_DB_STOCK=PATH_DATA+"\\sqlite\\dataset_paris_stock_adjusted.db"
PATH_DATA_DTS=PATH_DATA+"\\DTS_FULL\\"


CONNECTION TO SQLITE

In [3]:
if "con_stock" in locals():
        sio.close_connection(con_stock)
con_stock = sio.get_connection(str_db_path=PATH_DB_STOCK)

if "con_fwk" in locals():
        sio.close_connection(con_fwk)
con_fwk = sio.get_connection(str_db_path=PATH_DB_FWK)

my_session_maker = sessionmaker(bind=con_fwk)
session=my_session_maker()

table_stock="DS_PARIS_1D_ADJ_CLEAN"

GET DATA

In [4]:
dts_name="PARIS_TREND_1D_20D_V3"
multi_symbol="PARIS_STOCK"

df_base=sio.get_candles_to_df(session=session,con=con_stock, target_table=table_stock,tradable=True)
print(df_base[0:3])


DEBUG: SELECT CODE,OPEN_DATETIME, OPEN,HIGH,LOW,CLOSE,VOLUME FROM DS_PARIS_1D_ADJ_CLEAN can WHERE can.TIMEFRAME=1440 AND can.SK_SYMBOL IN (SELECT SK_SYMBOL FROM SYMBOL WHERE TRADABLE=1)   
                         OPEN    HIGH     LOW   CLOSE  VOLUME
CODE    OPEN_DATETIME                                        
ABCA.PA 2000-01-03     1.0267  1.0267  1.0267  1.0267  3490.0
        2000-01-04     1.0215  1.0215  1.0215  1.0215  4250.0
        2000-01-05     1.0382  1.0382  1.0382  1.0382  5960.0


In [ ]:
df_work=pd.DataFrame()
for code_value in df_base.index.get_level_values('CODE').unique():
    sub_df=df_base[df_base.index.get_level_values('CODE') == code_value]
    df_work_tmp = indic.add_indicators_to_df(con=con_fwk, df_in=sub_df, dts_name=dts_name,symbol=multi_symbol)
    df_work = pd.concat([df_work, df_work_tmp])
    
df_work.sort_index(inplace=True)
df_work.info() 

In [6]:
# print(df_base.index.get_level_values('CODE').unique())
# df_work[10000:10010]
# pd.set_option('display.max_columns', None)
# print(df_work.describe())

df_work.round(5).to_csv(
    PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_BASE_c", sep=",")

 FROM HERE TO LOAD THE BASE FILE

In [4]:
dts_name="PARIS_TREND_1D_20D_V3"
# dts_name="PARIS_TREND_1D_50D_V1"
multi_symbol="PARIS_STOCK"

df_work=pd.read_csv(PATH_DATA_DTS+dts_name+"_BASE.zip",sep=",",index_col=["OPEN_DATETIME","CODE"],parse_dates=["OPEN_DATETIME"])
df_work.head(5)

,,OPEN,HIGH,LOW,CLOSE,VOLUME,sma20,pos_sma20,sma50,sma200,pos_sma50,...,perf_atr14_5d,perf_adx14_5d,perf_adx14_neg_5d,perf_adx14_pos_5d,perf_adx14_dif_5d,avg_vol50,pos_avg_vol50,perf_williamsr_14_5d,williamsr_14_perf_10d,trix12
OPEN_DATETIME,CODE,,,,,,,,,,,,,,,,,,,,,
2010-04-26,AB.PA,12.98,12.98,12.20,12.68,62866.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-27,AB.PA,12.74,12.83,12.61,12.70,22370.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-28,AB.PA,12.70,12.70,12.41,12.50,8211.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-29,AB.PA,12.60,12.65,12.46,12.64,4676.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30,AB.PA,12.63,12.71,12.55,12.65,4470.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# drop the columns donchian20_hi,	donchian20_lo,	pos_donchian20_hi,	pos_donchian20_lo
# df_work.drop(columns=["donchian20_hi","donchian20_lo","pos_donchian20_hi","pos_donchian20_lo"],inplace=True)
# df_work.round(5).to_csv(
#     PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_BASE_a", sep=",")

In [8]:
#  extract 100 samples from 1000th line of COD AI.PA and write in a file for analyse
df_work.loc[(df_work.index.get_level_values('CODE') == "AI.PA")].iloc[5500:5600].to_csv(
    PATH_DATA_DTS+dts_name+"_BASE_AI_SAMPLES.csv", sep=",")

Check Dataframe

In [20]:
df_check=df_work[df_work['stdev20_1d'] > 100]
# df_check=df_check[df_check['ret_1d'] <= 2]
print(df_check.index.get_level_values('CODE').unique())
# print(df_check.info())
# df_check[df_check.index.get_level_values('CODE')=='AKW.PA']
# df_check=df_work[df_work.index.get_level_values('CODE')=='AI.PA']
# CATG
# mask = df_work['stdev20_1d'] > 100
# df_work.drop(df_work[mask].index, inplace=True)
# # print 10 first lines of df_check with code=AAA.PA 
# code="CBR.PA"#'CBE.PA', 'CNV.PA'
# print(df_check[df_check.index.get_level_values('CODE')==code][0:20])

# # print the graph of DGE.PA with candles mode
# df_work[df_work.index.get_level_values('CODE')==code].plot(y=["OPEN","HIGH","LOW","CLOSE"],kind='line',figsize=(15, 10))  
# plt.show()

# drop lines where CODE=CBE.PA
df_work.drop(df_work[df_work.index.get_level_values('CODE') == "CBE.PA"].index, inplace=True)
df_work.round(5).to_csv(
    PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_BASE_d", sep=",")



Index(['CBE.PA', 'CNV.PA'], dtype='object', name='CODE')


In [5]:
df_work = indic.drop_indicators_by_type(
    con=con_fwk, df_in=df_work, dts_name=dts_name, symbol=multi_symbol, ind_type=0)
list_label = indic.get_ind_list_by_type_for_dts(
    con=con_fwk, dts_name=dts_name, symbol_code=multi_symbol, ind_type=2)
print(type(list_label))
print(list_label)

<class 'pandas.core.frame.DataFrame'>
           LABEL
0  lab_perf_250d
1   lab_perf_20d
2   lab_perf_50d
3   lab_perf_10d


In [ ]:
#  prepare the dataframe for the indictor analysis
lab_split='lab_perf_50d'
df_work['CODE'] = df_work.index.get_level_values('CODE')
df_work=df_work.droplevel('CODE')
df_work.sort_index(inplace=True)

df_split=sm.split_df_by_label_strat(
        df_in=df_work, list_label=[lab_split], split_timeframe="M",random_split=False,split_strat=(80,10,10))

df_class = df_split['df_'+lab_split+'_train']
df_class.sort_index(inplace=True)

for label in list_label['LABEL']:
    df_class = balance.add_class_by_lab_nb_lines(
        df_in=df_class, str_label=label, nb_class=10, bool_replace_label=False)
    print(df_class[label+"_class"].value_counts())
# df_work.sort_index(inplace=True)
# df_class=balance.add_class_by_lab_nb_lines(df_in=df_work,str_label="lab_perf_20d",nb_class=5,bool_replace_label=False)
# df_class=balance.add_class_by_lab_nb_lines(df_in=df_class,str_label="lab_perf_50d",nb_class=5,bool_replace_label=False)
df_class.sort_index(inplace=True)
#  count lines per class in lab_perf_20d
# print(df_class['lab_perf_20d_class'].value_counts())
# print(df_class['lab_perf_50d_class'].value_counts())

In [ ]:
##### SAVE !!!!##############
df_class.round(5).to_csv(
    PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_CLASS_TRAIN", sep=",")

In [ ]:
# analysis of the datafarme with some indicators to find a filter
list_indic_test=['pos_sma20','pos_sma50','pos_sma200','pos_sma20_50','pos_sma50_200',
'pos_sma20_200','perf_sma200_10d','pos_ema20','pos_ema50','pos_ema200','pos_ema20_50',
'pos_ema50_200','pos_ema20_200','perf_ema200_10d','ret_5d','aroon14_dif','macd_dif',
'stdev20_1d','pos_stdev20_sma20','stoch14_dif','adx14_dif','trix12']

for label in list_label['LABEL']:#list_label['LABEL']:
    df_indic_test=df_class[label+"_class"].value_counts()
    df_indic_test.sort_index(inplace=True)
    # convert to dataframe
    df_indic_test=pd.DataFrame(df_indic_test)
    print(type(df_indic_test))
    # print(df_class[df_class['rsi14'] > 50][label+"_class"].value_counts().sort_index())
    df_indic_test['rsi14']=df_class[df_class['rsi14'] > 50][label+"_class"].value_counts().sort_index()
    df_indic_test['williamsr_14']=df_class[df_class['williamsr_14'] > -50][label+"_class"].value_counts().sort_index()
    for indic_name in list_indic_test:
        print(indic_name)
        df_indic_test[indic_name]=df_class[df_class[indic_name] > 0][label+'_class'].value_counts().sort_index()
    
    df_indic_test.round(5).to_csv(PATH_DATA_DTS+"PARIS_TREND_1D_20D_indic_test_"+label, sep=",")
        
# print(df_indic_test)

# print nb lines with pos_sma200>0 and pos_sma200<0
# print(df_class[df_class['pos_sma50_200'] > 0].shape)
# print(df_class[df_class['pos_sma50_200'] < 0].shape)

# # print nb linesper class with pos_sma200>0 and pos_sma200<0
# print(df_class[df_class['pos_sma50_200'] > 0]['lab_perf_20d_class'].value_counts().sort_index())
# print(df_class[df_class['pos_sma50_200'] < 0]['lab_perf_20d_class'].value_counts().sort_index())

# print(df_class[df_class['pos_sma50_200'] > 0]['lab_perf_50d_class'].value_counts().sort_index())
# print(df_class[df_class['pos_sma50_200'] < 0]['lab_perf_50d_class'].value_counts().sort_index())

In [6]:
#  drop label lab_perf_250d and lab_perf_10d
df_work.drop(columns=['lab_perf_250d','lab_perf_10d'],inplace=True)

In [7]:
#  filter the dataframe with pos_sma20_200>=0
df_work=df_work[df_work['pos_sma20_200']>=0]
df_work.sort_index(inplace=True)
df_work.round(5).to_csv(
    PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_filtered", sep=",")

In [10]:
df_work['CODE'] = df_work.index.get_level_values('CODE')
df_work=df_work.droplevel('CODE')
df_work.sort_index(inplace=True)
df_work.head(5)

,OPEN,HIGH,LOW,CLOSE,VOLUME,pos_sma20,pos_sma50,pos_sma200,pos_sma50_200,pos_sma20_50,...,perf_atr14_5d,perf_adx14_5d,perf_adx14_neg_5d,perf_adx14_pos_5d,perf_adx14_dif_5d,pos_avg_vol50,perf_williamsr_14_5d,williamsr_14_perf_10d,trix12,CODE
OPEN_DATETIME,,,,,,,,,,,,,,,,,,,,,
1991-10-09,0.703,0.703,0.703,0.703,1792.0,-0.03047,-0.02040,0.12800,0.15150,0.01038,...,0.01714,0.03223,2.51985,-2.51986,-5.03971,0.15837,0.00000,0.00000,-0.07907,BOI.PA
1991-10-10,0.703,0.703,0.703,0.703,864.0,-0.02844,-0.02040,0.12678,0.15025,0.00827,...,-0.12066,0.12260,10.51980,-10.51981,-21.03961,0.07732,-31.91489,0.00000,-0.10139,BOI.PA
1991-10-11,0.703,0.703,0.703,0.703,17920.0,-0.02578,-0.02040,0.12556,0.14900,0.00552,...,-0.30964,0.11071,-0.00001,-0.00001,0.00000,1.57693,0.00000,-5.48523,-0.11993,BOI.PA
1991-10-14,0.703,0.703,0.703,0.703,13984.0,-0.02311,-0.02033,0.12434,0.14768,0.00284,...,-0.30964,0.10024,-0.00001,-0.00001,0.00000,1.20180,0.00000,-10.97046,-0.13459,BOI.PA
1991-10-15,0.703,0.703,0.703,0.703,2208.0,-0.02105,-0.02033,0.12316,0.14647,0.00074,...,-0.30964,0.09098,-0.00001,-0.00001,0.00000,0.18969,0.00000,-10.97046,-0.14550,BOI.PA


In [ ]:
label = "lab_perf_20d"
# algo_studied = "XG_BOOST_CLASS"
dts_name="PARIS_TREND_1D_50D_V3"

# df_work_lab = indic.drop_indicators_not_selected(con=con_fwk, df_in=df_work, dts_name=dts_name, symbol=multi_symbol,label=lab_studied,algo=algo_studied)

df_split=sm.split_df_by_label_strat(
    df_in=df_work, list_label=[label], split_timeframe="M",random_split=False,split_strat=(80,10,10))
df_selected = df_split['df_'+label+'_train']
df_valid = df_split['df_'+label+'_valid']
df_confirm = df_split['df_'+label+'_confirm']
df_selected.sort_index(inplace=True)
df_valid.sort_index(inplace=True)
df_confirm.sort_index(inplace=True)

print(f"selected: {df_selected.shape=} valid: {df_valid.shape=} confirm: {df_confirm.shape=}")


selected: df_selected.shape=(486879, 136) valid: df_valid.shape=(102506, 136) confirm: df_confirm.shape=(120468, 136)


In [14]:
label = "lab_perf_50d"
df_class=balance.add_class_by_lab_nb_lines(df_in=df_selected,str_label=label,nb_class=5,bool_replace_label=False)
min_max_lab_by_class = df_class.groupby(label+'_class')[label].agg(['min', 'max'])
print(min_max_lab_by_class)

                        min      max
lab_perf_50d_class                  
0                  -0.79047 -0.05826
1                  -0.05825  0.00124
2                   0.00125  0.05301
3                   0.05302  0.12161
4                   0.12162  3.09581


In [ ]:
label = "lab_perf_20d"
df_class=balance.add_class_by_lab_nb_lines(df_in=df_selected,str_label=label,nb_class=5,bool_replace_label=True)
df_class.sort_index(inplace=True)
# categ_50={0:[-1,-0.05826],1:[-0.05826,0.00124],2:[0.00124,0.05301],3:[0.05301,0.12161],4:[0.12161,5]}
categ_20={0:[-1,-0.04141],1:[-0.04141,-0.00522],2:[-0.00522,0.02439],3:[0.02439,0.06687],4:[0.06687,5]}
df_class_val=balance.add_lab_by_class(df_in=df_valid,str_label=label, categ=categ_20,bool_replace_label=True) # categ
df_class_val.sort_index(inplace=True)
df_class_conf=balance.add_lab_by_class(df_in=df_confirm,str_label=label, categ=categ_20,bool_replace_label=True) # categ
df_class_conf.sort_index(inplace=True)
print(df_class.loc[:, label].dropna().iloc[[0, -1]])
print(df_class_val.loc[:, label].dropna().iloc[[0, -1]])
print(df_class_conf.loc[:, label].dropna().iloc[[0, -1]])
# df_class_clean=df_class.drop(['OPEN','HIGH','LOW','CLOSE','VOLUME','lab_perf_125d','lab_perf_20d','lab_perf_50d'],axis=1)
data = df_class[label]
print(data.value_counts().sort_index())
data_val = df_class_val[label]
print(data_val.value_counts().sort_index())
data_conf = df_class_conf[label]
print(data_conf.value_counts().sort_index())
# min_max_lab_by_class = df_class.groupby(label+'_class')[label].agg(['min', 'max'])
# print(min_max_lab_by_class)

#                         min      max
# lab_perf_20d_class                  
# 0                  -0.87743 -0.05172
# 1                  -0.05171 -0.00868
# 2                  -0.00867  0.02272
# 3                   0.02273  0.07058
# 4                   0.07059  3.82176

In [ ]:
df_class.round(5).to_csv(
    PATH_DATA+"\DTS_FULL\PARIS_TREND_1D_20D_V3_TRAIN.zip", sep=",")
df_class_val.round(5).to_csv(
    PATH_DATA+"\DTS_FULL\PARIS_TREND_1D_20D_V3_VAL.zip", sep=",")
df_class_conf.round(5).to_csv(
    PATH_DATA+"\DTS_FULL\PARIS_TREND_1D_20D_V3_CONF.zip", sep=",")

In [ ]:
list_feat = df_class.columns.values.tolist()
list_feat.remove(label)
df_x_train, col_y_train = sm.split_df_x_y(
    df_in=df_class, list_features=list_feat, str_label=label, drop_na=True)
df_x_val, col_y_val = sm.split_df_x_y(
    df_in=df_class_val, list_features=list_feat, str_label=label, drop_na=True)
df_x_conf, col_y_conf = sm.split_df_x_y(
    df_in=df_class_conf, list_features=list_feat, str_label=label, drop_na=True)



In [ ]:
dts_name="PARIS_TREND_1D_50D_V3"
multi_symbol="PARIS_STOCK"
label = "lab_perf_50d"

df_class=pd.read_csv(PATH_DATA_DTS+dts_name+"_TRAIN.zip",sep=",",index_col=["OPEN_DATETIME"],parse_dates=["OPEN_DATETIME"])
df_class.dropna(subset=[label], inplace=True)
df_class.sort_index(inplace=True)

df_class_val=pd.read_csv(PATH_DATA_DTS+dts_name+"_VAL.zip",sep=",",index_col=["OPEN_DATETIME"],parse_dates=["OPEN_DATETIME"])
df_class_val.dropna(subset=[label], inplace=True)
df_class_val.sort_index(inplace=True)

df_class_conf=pd.read_csv(PATH_DATA_DTS+dts_name+"_CONF.zip",sep=",",index_col=["OPEN_DATETIME"],parse_dates=["OPEN_DATETIME"])
df_class_conf.dropna(subset=[label], inplace=True)
df_class_conf.sort_index(inplace=True)

categ_50={0:[-1,-0.05826],1:[-0.05826,0.00124],2:[0.00124,0.05301],3:[0.05301,0.12161],4:[0.12161,5]}
df_class=balance.add_lab_by_class(df_in=df_class,str_label=label, categ=categ_50,bool_replace_label=True) # categ
df_class.sort_index(inplace=True)
df_class_val=balance.add_lab_by_class(df_in=df_class_val,str_label=label, categ=categ_50,bool_replace_label=True) # categ
df_class_val.sort_index(inplace=True)
df_class_conf=balance.add_lab_by_class(df_in=df_class_conf,str_label=label, categ=categ_50,bool_replace_label=True) # categ
df_class_conf.sort_index(inplace=True)

print(df_class[label].value_counts().sort_index()) # min 97366 
print(df_class_val[label].value_counts().sort_index()) # min 14774
print(df_class_conf[label].value_counts().sort_index()) # min 20566

lab_perf_50d
0.0    97366
1.0    97375
2.0    97372
3.0    97388
4.0    97378
Name: count, dtype: int64
lab_perf_50d
0.0    29424
1.0    21370
2.0    20005
3.0    16891
4.0    14774
Name: count, dtype: int64
lab_perf_50d
0.0    30933
1.0    24911
2.0    21353
3.0    20566
4.0    22705
Name: count, dtype: int64


In [ ]:
# save the 3 df in pkl format
df_class.to_pickle(PATH_DATA_DTS+dts_name+"_TRAIN.pkl")
df_class_val.to_pickle(PATH_DATA_DTS+dts_name+"_VAL.pkl")
df_class_conf.to_pickle(PATH_DATA_DTS+dts_name+"_CONF.pkl")

In [11]:
dts_name="PARIS_TREND_1D_50D_V3"
multi_symbol="PARIS_STOCK"
label = "lab_perf_50d"

# load the 3 pkl files in 3 df
df_class=pd.read_pickle(PATH_DATA_DTS+dts_name+"_TRAIN.pkl")
df_class_val=pd.read_pickle(PATH_DATA_DTS+dts_name+"_VAL.pkl")
df_class_conf=pd.read_pickle(PATH_DATA_DTS+dts_name+"_CONF.pkl")

In [ ]:
# USELESS !!
list_feat = df_class.columns.values.tolist()
list_feat.remove(label)
df_x_train, col_y_train = sm.split_df_x_y(
    df_in=df_class, list_features=list_feat, str_label=label, drop_na=True)

# X = df_class.drop(label, axis=1, inplace=False)
# y = df_class[label]
method = RandomUnderSampler(sampling_strategy={0:97300,1:97300,2:97300,3:97300,4:97300}) # 97300 pour lab 20 et 50
df_x_train, col_y_train=  method.fit_resample(df_x_train, col_y_train)
print(col_y_train.value_counts().sort_index())

# X = df_class_val.drop(label, axis=1, inplace=False)
# y = df_class_val[label]
df_x_train, col_y_train = sm.split_df_x_y(
    df_in=df_class, list_features=list_feat, str_label=label, drop_na=True)
method = RandomUnderSampler(sampling_strategy={0:16800,1:16800,2:16800,3:16800,4:16800}) # 16800 pour lab 20 et 45000 pour lab 50
df_x_val, col_y_val=  method.fit_resample(X, y)
print(col_y_val.value_counts().sort_index())
# df_x_val, col_y_val = sm.split_df_x_y(
    # df_in=df_class_val, list_features=list_feat, str_label=label, drop_na=True)

In [12]:
label = "lab_perf_50d"

# for lab_perf_20d
list_rem=["lab_perf_20d","lab_perf_50d","CODE","OPEN","HIGH","LOW","CLOSE","VOLUME","lab_perf_125d"]
# list_rem.extend(["aroon14_down_lag_5d", "aroon14_up", "aroon14_up_lag_5d","sma5_rsi14",
# "pos_sma10","pos_sma5_50","pos_sma5_200","pos_sma10_50","pos_sma10_200"
# ,"pos_ema10","perf_ema10_5d","pos_ema20","pos_ema50","perf_ema50_5d"
# ,"pos_ema200","perf_pos_ema200_5d","perf_ema200_5d","perf_ema200_10d"
# ,"pos_ema5_10","pos_ema5_20","pos_ema5_50","pos_ema5_200","pos_ema10_20"
# ,"pos_ema10_50","pos_ema10_200","pos_ema20_50","macd_fast","trix12"
# ,"perf_sma_20_5d","perf_adx14_neg_5d","perf_adx14_pos_5d,aroon14_down"
# ,"perf_adx14_pos_5d",    "perf_rsi14_10d",    "pos_top50",    "stdev20_1d",
#     "ret_20d",    "perf_sma20_10d",    "pos_bot20",    "pos_bb20_lo_lag_5d",
#     "pos_bot20_lag_5d",    "pos_sma50",    "stoch14_dif",    "pos_rsi14_sma5",
#     "pos_bb20_lo",    "perf_rsi14_5d",    "pos_bb20_hi_lag_5d",    "pos_bb20_hi",
#     "aroon14_down",    "perf_pos_sma20_10d",    "perf_pos_sma50_10d",    "pos_top20",
#     "pos_top20_lag_5d",    "perf_pos_sma10_10d",    "perf_sma10_10d",
#     "perf_pos_sma200_10d",    "ret_1d",    "perf_sma5_5d",    "perf_ema50_10d",
#     "pos_sma20",    "perf_ema10_10d",    "ret_5d",    "perf_sma20_5d",    "perf_pos_ema200_10d",
#     "perf_pos_ema20_5d",    "perf_pos_ema20_10d",    "perf_pos_ema10_5d",
#     "perf_ema5_5d",    "perf_ema20_10d",    "perf_pos_ema10_10d",
#     "ret_10d",    "pos_sma5_20",    "perf_sma_200_5d",    "pos_sma5",    "pos_sma10_20",
#     "perf_sma_50_5d",    "perf_sma50_5d",    "perf_sma200_5d",    "perf_sma_10_5d",
#     "pos_ema5",    "pos_sma5_10",    "perf_ema20_5d",    "perf_sma10_5d","pos_sma50_200","pos_sma20_200"
# ])

# for lab_perf_50d
list_rem.extend(["pos_sma10","pos_sma5_50","pos_sma5_200","pos_sma10_50","pos_sma10_200",
"pos_ema10","perf_ema10_5d","pos_ema20","pos_ema50","perf_ema50_5d","pos_ema200",
"perf_ema200_5d","perf_ema200_10d","pos_ema5_10","pos_ema5_20","pos_ema5_50",
"pos_ema5_200","pos_ema10_20","pos_ema10_50","pos_ema10_200","pos_ema20_50",
"trix12","perf_sma_20_5d",    "ret_20d",    "perf_sma20_10d",       "pos_sma50",  
     "perf_pos_sma50_10d",  "perf_sma10_10d",    "perf_pos_sma200_10d", 
	   "perf_sma5_5d",  	   "perf_ema50_10d",    "pos_sma20",  	"perf_ema10_10d",  
	    "perf_sma20_5d",   		"perf_pos_ema200_10d",    "perf_ema5_5d",  
	"perf_ema20_10d",     "ret_10d", 	"pos_sma5_20", 	    "pos_sma10_20",
    "perf_sma_50_5d",    "perf_sma50_5d",   	    "perf_sma_10_5d",
    "pos_ema5",  	"pos_sma5_10",  	"perf_ema20_5d", 	"perf_sma10_5d",	"pos_sma20_200",
    "pos_rsi14_sma20","tr_atr14","pos_avg_vol14","pos_top20",
    "perf_adx14_dif_5d","pos_ema50_perf_5d","perf_stoch14_signal_5d",
    "pos_bb20_hi_lag_5d","perf_rsi14_10d","pos_top20_lag_5d",
    "perf_adx14_neg_5d","perf_stoch14_5d","aroon14_up_lag_5d",
    "perf_pos_sma20_10d","perf_adx14_pos_5d","aroon14_down_lag_5d",
    "perf_williamsr_14_5d","aroon14_up","aroon14_down",
    "perf_sma200_5d","perf_stoch14_dif_5d","stdev20_1d",
    "perf_sma_200_5d","pos_rsi14_sma5","perf_pos_sma10_10d",
    "perf_rsi14_5d","stoch14_dif","perf_pos_ema200_5d",
    "perf_pos_ema20_10d","ret_5d","perf_pos_ema10_5d",
    "perf_pos_ema20_5d","perf_pos_ema10_10d","pos_sma5","ret_1d"
])

list_feat = df_class.columns.values.tolist()
# remove list_rem from list_feat, don't crash if not in list

print(f"{list_feat.__len__()=}")
list_feat = [x for x in list_feat if x not in list_rem]

print(f"{list_feat.__len__()=}")

df_x_train, col_y_train = sm.split_df_x_y(
    df_in=df_class,list_features=list_feat, str_label=label, drop_na=True)
# df_x_train.drop(list_rem, axis=1, inplace=True)
print(df_x_train.shape)

list_feat.__len__()=136
list_feat.__len__()=45
(486879, 45)


Correlation

In [ ]:

corr_train = df_x_train.corr()
fig, ax = plt.subplots(figsize=(6, 6))
corr_train.replace(1,0,inplace=True)
# corr_train=corr_train.applymap(lambda x : None if x< 0.8 and x>-0.8 else x)
corr_train.dropna(axis=0,how='all',inplace=True)
corr_train.dropna(axis=1,how='all',inplace=True)

corr_train_check=corr_train[corr_train >0.8]
corr_train_check.head()
sns.heatmap(corr_train_check, annot=False, cmap='coolwarm', vmin=-1, vmax=1, ax=ax)

# save the correlation matrix in a file
corr_train.to_csv(PATH_DATA_DTS+dts_name+"_CORR_TRAIN.csv", sep=",")


In [ ]:
# print pairs of features with correlation > 0.9
corr_train_unstack = corr_train.unstack()
corr_train_unstack = corr_train_unstack[corr_train_unstack > 0.9]
corr_train_unstack = corr_train_unstack[corr_train_unstack < 1]
# save the whole list in a file
corr_train_unstack.to_csv(PATH_DATA_DTS+dts_name+"_CORR_TRAIN_UNSTACK.csv", sep=",")

BORUTA

In [ ]:
rnd_key=28
n_jobs=3

method = RandomUnderSampler(sampling_strategy={0:97300,1:97300,2:97300,3:97300,4:97300}) # 97300
df_x_boruta, col_y_boruta=  method.fit_resample(df_x_train, col_y_train)


print(f"{df_x_boruta.shape=}")
print(col_y_boruta.value_counts().sort_index())

del df_x_train
del col_y_train

rf_bo = RandomForestClassifier(n_estimators=100, n_jobs=n_jobs, max_depth=7)
boruta_selector = BorutaPy(rf_bo, n_estimators='auto',
                           verbose=1, random_state=int(rnd_key),max_iter=100)
boruta_selector.fit(df_x_boruta.values, col_y_boruta.values)
selected_features = df_x_boruta.columns[boruta_selector.support_]
print(selected_features)

not_selected_features = set(df_x_boruta.columns) - set(selected_features)
print(not_selected_features)

#  print the ranking of the features with details
print(boruta_selector.ranking_)
print(boruta_selector.support_)

# save this in a file
boruta_result=pd.DataFrame()
boruta_result['features']=df_x_boruta.columns
boruta_result['rank']=boruta_selector.ranking_
boruta_result['support']=boruta_selector.support_
boruta_result.to_csv(PATH_DATA_DTS+dts_name+"_BORUTA_RANK.csv", sep=",")


In [11]:
# print le value counts of the label in the training set and the validation set
print(df_class[label].value_counts().sort_index())
print(df_class_val[label].value_counts().sort_index())


lab_perf_50d
0.0    97366
1.0    97375
2.0    97372
3.0    97388
4.0    97378
Name: count, dtype: int64
lab_perf_50d
0.0    29424
1.0    21370
2.0    20005
3.0    16891
4.0    14774
Name: count, dtype: int64


In [13]:
pd.set_option('display.max_columns', None)
# print(df_x_train.describe())

# à revoir !!
# cas des valeurs proches de 0
# faire avec le min et max des valeurs
# def round_to_significant_digits(x, digits):
#     if x == 0:
#         return 0
#     else:
#         # Calculate the number of significant digits
#         rounded_value = round(x, digits - int(np.floor(np.log10(abs(x)))) - 1)
#         # Adjust the rounding for values between -1 and 1
#         if abs(x) < 1:
#             return round(rounded_value, digits)
#         else:
#             return rounded_value

def round_to_significant_digits(df, digits):
    list_zero=["rsi14","sma5_rsi14","sma20_rsi14","aroon14_up","aroon14_down","aroon14_dif",
"stoch14","stoch14_signal","stoch14_dif","adx14","adx14_neg","adx14_pos",
"perf_rsi14_5d","perf_rsi14_10d","aroon14_up_lag_5d","aroon14_down_lag_5d",
"aroon14_dif_lag_5d",	"perf_stoch14_5d","perf_stoch14_signal_5d",
"perf_stoch14_dif_5d","perf_adx14_neg_5d",	"perf_adx14_pos_5d",	
"perf_williamsr_14_5d","williamsr_14_perf_10d","williamsr_14"	]
    df_temp=df.copy()
    for col in df_temp.columns:
        if col in list_zero:
            df_temp[col]=df_temp[col].apply(lambda x:round(x,digits-2))
        else:
            df_temp[col]=df_temp[col].apply(lambda x:round(x,digits))
    return df_temp


df_x_train, col_y_train = sm.split_df_x_y(
    df_in=df_class, list_features=list_feat, str_label=label, drop_na=True)

df_x_val, col_y_val = sm.split_df_x_y(
    df_in=df_class_val, list_features=list_feat, str_label=label, drop_na=True)


method = RandomUnderSampler(sampling_strategy={0:97300,1:97300,2:97300,3:97300,4:97300}) # 97300 pour lab 20 et 97300 pour lab 50
df_x_train, col_y_train=  method.fit_resample(df_x_train, col_y_train)

method = RandomUnderSampler(sampling_strategy={0:14750,1:14750,2:14750,3:14750,4:14750}) # 16800 pour lab 20 et 14750 pour lab 50
df_x_val, col_y_val=  method.fit_resample(df_x_val, col_y_val)

print(f"{df_x_train.shape=} {df_x_val.shape=} {col_y_train.size=} {col_y_val.size=}")

nb_digits = 2
# round df_x_train and df_x_val
# df_x_train_r = df_x_train.map(lambda x:round_to_significant_digits(x,nb_digits),na_action='ignore')
# df_x_val_r = df_x_val.map(lambda x:round_to_significant_digits(x,nb_digits),na_action='ignore')

df_x_train_r = round_to_significant_digits(df_x_train, nb_digits)
df_x_val_r = round_to_significant_digits(df_x_val, nb_digits)

# save objects in a pickle files
joblib.dump(df_x_train_r, PATH_DATA_DTS+dts_name+"_X_TRAIN_FILT_R"+str(nb_digits)+".pkl")
joblib.dump(col_y_train, PATH_DATA_DTS+dts_name+"_Y_TRAIN_FILT_R"+str(nb_digits)+".pkl")
joblib.dump(df_x_val_r, PATH_DATA_DTS+dts_name+"_X_VAL_FILT_R"+str(nb_digits)+".pkl")
joblib.dump(col_y_val, PATH_DATA_DTS+dts_name+"_Y_VAL_FILT_R"+str(nb_digits)+".pkl")



df_x_train.shape=(486500, 45) df_x_val.shape=(73750, 45) col_y_train.size=486500 col_y_val.size=73750


['G:\\Python\\Data\\DTS_FULL\\PARIS_TREND_1D_50D_V3_Y_VAL_FILT_R2.pkl']

In [3]:
dts_name="PARIS_TREND_1D_50D_V3"
multi_symbol="PARIS_STOCK"
label = "lab_perf_50d"
rounded_digits="2"
# load the 4 pkl files saved before
df_x_train_r = joblib.load(PATH_DATA_DTS+dts_name+"_X_TRAIN_FILT_R"+rounded_digits+".pkl")
col_y_train = joblib.load(PATH_DATA_DTS+dts_name+"_Y_TRAIN_FILT_R"+rounded_digits+".pkl")
df_x_val_r = joblib.load(PATH_DATA_DTS+dts_name+"_X_VAL_FILT_R"+rounded_digits+".pkl")
col_y_val = joblib.load(PATH_DATA_DTS+dts_name+"_Y_VAL_FILT_R"+rounded_digits+".pkl")



In [ ]:
# drop from df_x_train_r and df_x_val_r the columns pos_sma50_200 and pos_sma20_200
df_x_train_r.drop(columns=["pos_sma50_200","pos_sma20_200"],inplace=True)
df_x_val_r.drop(columns=["pos_sma50_200","pos_sma20_200"],inplace=True)

print(f"{rounded_digits=}")

# save the new dataframes in pkl files
joblib.dump(df_x_train_r, PATH_DATA_DTS+dts_name+"_X_TRAIN_FILT_R"+rounded_digits+".pkl")
joblib.dump(df_x_val_r, PATH_DATA_DTS+dts_name+"_X_VAL_FILT_R"+rounded_digits+".pkl")

# save list of columns of df_x_train_r in an info file   
with open(PATH_DATA_DTS+dts_name+"_X_TRAIN_FILT_R"+rounded_digits+"_INFO.txt", "w") as f:
    for s in df_x_train_r.columns:
        f.write(str(s) +"\n")

In [4]:

print(f"{df_x_train_r.shape=} {df_x_val_r.shape=} {col_y_train.size=} {col_y_val.size=}")
# print(df_x_train_r.head(10))
# print(col_y_val.head(10))

df_x_train_r.shape=(486500, 45) df_x_val_r.shape=(73750, 45) col_y_train.size=486500 col_y_val.size=73750


In [ ]:
# print df_x_train_r describe() but with all the columns
pd.set_option('display.max_columns', None)
print(df_x_train_r.describe())

First test Random Forest

In [ ]:
# BLOCK ANTIMALWARE FOR SOME HOURS
n_jobs=3


col_y_train_int = col_y_train.to_numpy().astype(int)
col_y_val_int = col_y_val.to_numpy().astype(int)

# eval_set=[(df_x_val, col_y_val_int)]

scoring = {
    'accuracy': metrics.make_scorer(metrics.accuracy_score),
    'precision_weighted': metrics.make_scorer(metrics.precision_score, average='weighted'),
    'recall_weighted': metrics.make_scorer(metrics.recall_score, average='weighted'),
    'f1_weighted': metrics.make_scorer(metrics.f1_score, average='weighted')
}

time_split = TimeSeriesSplit(n_splits=5)

# np.linspace(0.1, 0.5, 5, endpoint=True) # [0,0.01,0.02,0.03,0.04,0.05,0.06,0.075,0.1,0.12,0.15,0.2,0.25,0.3,0.4,0.5]
grid = {
    # number of trees in the foreset
    'n_estimators': np.linspace(50, 250,5, endpoint=True,dtype=int),
    # max number of features considered for splitting a node
    'max_features': ['sqrt'],
    # max number of levels in each decision tree
    'max_depth': np.linspace(4, 7, 4, endpoint=True,dtype=int),
    # min number of data points placed in a node before the node is split
    'min_samples_split': [0.075,0.1,0.125,0.15,0.2,0.25,0.3],
    # min number of data points allowed in a leaf node
    'min_samples_leaf': [0.1,0.15,0.2,0.25],# max 0.3
    # min weighted fraction of the sum total of weights
    'min_weight_fraction_leaf':[0.01,0.03,0.05,0.075,0.1,0.12,0.15,0.2,0.25,0.3],
    #min impurity decrease A node will be split if this split induces a decrease of the impurity greater than or equal to this value.
    #'min_impurity_decrease':[0,0.01],
    'class_weight':['balanced']#[None,'balanced']
    }

rf = RandomForestClassifier()

rf_fitted = modmgr.search_cv_fit_report(estimator=rf, params=grid,cv=time_split,n_jobs=n_jobs,
                                        x_train=df_x_train, y_train=col_y_train_int, 
                                          n_iter=20, n_top=5, method="random", 
                                          scoring=metrics.make_scorer(metrics.f1_score, average='weighted'))



LIGHTGBM

In [33]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

def custom_penalty_score(y_true, y_pred):
    # print(f"{y_true.shape=} {y_pred.shape=}")
    if isinstance(y_true, lgb.Dataset):
        y_true = y_true.get_label()
    # Convert predicted probabilities to class labels
    if y_pred.ndim == 1:
        # raise ValueError(f"Expected 2D array for y_pred, got 1D array with shape {y_pred.shape}")
        y_pred_labels = y_pred
    else :
        y_pred_labels = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels
    if y_true.shape[0] != y_pred_labels.shape[0]:
        raise ValueError(f"Shape mismatch: y_true has shape {y_true.shape} but y_pred_labels has shape {y_pred_labels.shape}")
  
    error = (y_true - y_pred_labels) ** 2
    return 'custom_penalty', error.mean(), False  # Return the mean squared error (lower is better)

# Create a scorer object
custom_scorer = metrics.make_scorer(custom_penalty_score, is_higher_better=False)

def print_eval_metric(env):
    # env is a callback environment that contains information about the training process
    result = env.evaluation_result_list
    for item in result:
        metric_name, dataset_name, metric_value, is_higher_better = item
        print(f"Iteration {env.iteration}: {dataset_name} - {metric_name} = {metric_value}")

def calculate_tree_depth(tree):
    if 'leaf_index' in tree:
        return 0
    else:
        left_depth = calculate_tree_depth(tree['left_child'])
        right_depth = calculate_tree_depth(tree['right_child'])
        return 1 + max(left_depth, right_depth)


In [ ]:


dts_name="PARIS_TREND_1D_50D_V3"
multi_symbol="PARIS_STOCK"
label = "lab_perf_50d"


num_threads=1
n_jobs=1
device_type='gpu'
stopping_rounds=20
n_iter=30
n_splits=4

time_split = TimeSeriesSplit(n_splits=n_splits)

rng = np.random.default_rng()

params = {
    'num_class': [5],
    'learning_rate': [0.01],#0.005
    'n_estimators': [1000],#np.linspace(400, 800,5, endpoint=True,dtype=int),
    'max_depth': [13],#np.linspace(11,15,3, endpoint=True,dtype=int),
    'num_leaves': [511],#1023,1536],#np.linspace(15, 60,10, endpoint=True,dtype=int),
    'min_data_in_leaf': [100],
    'feature_fraction': [0.2,0.4,0.6,0.8],
    'lambda_l1': np.exp(rng.uniform(np.log(1e-9), np.log(1000),100)) ,#[0,0.5,1.0,10],#np.exp(np.random.uniform(np.log(1e-9), np.log(1000)),10)
    'lambda_l2': np.exp(rng.uniform(np.log(1e-9), np.log(1000),100)),#[0,0.5,1.0,10]
    #'metric': ['multi_logloss']#['auc_mu','multi_logloss','multi_error','cross_entropy'],
}


clf=lgb.LGBMClassifier(objective='multiclass',metric='multi_logloss',device_type=device_type,verbosity=-1,
                       seed=np.random.default_rng().integers(0,10000))

#  now using search_cv_fit_report
# callbacks=[lgb.early_stopping(stopping_rounds=stopping_rounds)]
grid_model = modmgr.search_cv_fit_report(estimator=clf, params=params,cv=n_splits,n_jobs=n_jobs,
                                        x_train=df_x_train_r, y_train=col_y_train,
                                        eval_set=[(df_x_val_r, col_y_val)],
                                        n_iter=n_iter, n_top=10, method="random",
                                        scoring=custom_scorer#metrics.make_scorer(accuracy_score)#(metrics.f1_score, average='weighted')
                                        ,callbacks=[lgb.early_stopping(stopping_rounds=stopping_rounds)]
                                        ,verbose=1
                                        )


print(grid_model.best_params_)

# model = lgb.cv(params,
#                   lgb_ds_train,
#                   num_boost_round=numround,
#                 valid_sets=[lgb_ds_val],
#                 callbacks=[lgb.early_stopping(stopping_rounds=stopping_rounds)],folds=time_split,shuffle=False,
#                 seed=np.random.default_rng().integers(0,10000)
#                   )

# y_pred = grid_model.predict(df_x_val_r)
# y_pred = np.argmax(y_pred, axis=1)
# accuracy = accuracy_score(col_y_val, y_pred)
# print(f'Accuracy: {accuracy}')
# print(classification_report(col_y_val, y_pred))

In [38]:
import itertools
device_type='gpu'
results_df = pd.DataFrame(columns=['params','accuracy_train', 'accuracy_val','penalty_score',
                                   'precision','recall','best_iter','seed','max_max_depth','avg_max_depth'])

params = {
    'learning_rate': [0.001],  # 0.005
    'max_depth': [10],  # np.linspace(11,15,3, endpoint=True,dtype=int),
    'num_leaves': [7,11,15,19,23],  # 1023,1536],#np.linspace(15, 60,10, endpoint=True,dtype=int),
    'min_data_in_leaf': [25],
    'feature_fraction': [ 0.05,0.15],
    'lambda_l1': [0,1],#np.exp(np.random.default_rng().uniform(np.log(1e-4), np.log(10), 20)),
    'lambda_l2': [0,1],#np.exp(np.random.default_rng().uniform(np.log(1e-4), np.log(10), 20)),
}

# Generate all combinations of parameters
keys, values = zip(*params.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# Print the list of parameter combinations
print(f"{combinations.__len__()=}")
for combo in combinations[:2]:  # Print only the first 3 combinations for brevity
    print(combo)

# params=[
#     {'num_leaves': 511, 'min_data_in_leaf': 100, 'max_depth': 11, 'learning_rate': 0.01, 'lambda_l2': 0.025, 'lambda_l1': 0.01, 'feature_fraction': 0.4},
    # {'num_leaves': 1023, 'min_data_in_leaf': 100, 'max_depth': 15, 'learning_rate': 0.01, 'lambda_l2': 0.5, 'lambda_l1': 0.25, 'feature_fraction': 0.6}
#        ]
seed=np.random.default_rng().integers(0,10000)
print(f"{seed=}")
# loop over the parameters
for param in combinations:
    try:
        
        print(f"Try : {param}")
        clf=lgb.LGBMClassifier(objective='multiclass',metric='custom',device_type=device_type,verbosity=-1,
                            seed=seed, num_class=5,n_estimators=1000,#np.random.default_rng().integers(0,10000),
                            **param)
        # train the model with the best parameters and eval set without log
        clf.fit(df_x_train_r, col_y_train, eval_set=[(df_x_val_r, col_y_val)],eval_metric=custom_penalty_score,
                callbacks=[lgb.early_stopping(stopping_rounds=20)])#print_eval_metric

        # print(f"Best iteration: {clf.best_iteration_}")

        y_pred = clf.predict(df_x_train_r)
        accuracy_train = accuracy_score(col_y_train, y_pred)
        penalty_score=custom_penalty_score(col_y_train, y_pred)
        # print(f'Accuracy Train: {accuracy_train} Penalty Train: {penalty_score}')

        y_pred = clf.predict(df_x_val_r)
        accuracy_val = accuracy_score(col_y_val, y_pred)
        penalty_score=custom_penalty_score(col_y_val, y_pred)
        # print(f'Accuracy Val: {accuracy_val} Penalty Val: {penalty_score}')
        
        # Extract precision values
        precision_vals = precision_score(col_y_val, y_pred, average=None)
        precision_str ='precision:'+'-'.join([f'{i}:{precision_vals[i]:.2f}' for i in range(len(precision_vals))])
        
        # Extract recall values
        recall_vals = recall_score(col_y_val, y_pred, average=None)
        recall_str = 'recall:'+'-'.join([f'{i}:{recall_vals[i]:.2f}' for i in range(len(recall_vals))])

        # print(classification_report(col_y_val, y_pred))
        # print the confusion matrix
        print(metrics.confusion_matrix(col_y_val, y_pred))

        # Calculate the depth of each tree
        booster = clf.booster_
        tree_info = booster.dump_model()["tree_info"]
        # print(tree_info)
        max_depths = [calculate_tree_depth(tree["tree_structure"]) for tree in tree_info]
        avg_max_depth = sum(max_depths) / len(max_depths)
        max_max_depth = max(max_depths)
        # print(f"Max depth of trees: {avg_max_depth=} {max_max_depth=}")


        # save the values of param and accuracy_val in a dataframe
        results_df = pd.concat([results_df, 
                                pd.DataFrame([{'params': param,'accuracy_train':round(accuracy_train,3),
                                                'accuracy_val': round(accuracy_val,3),'penalty_score':round(penalty_score[1],3),
                                                'precision':precision_str,'recall':recall_str,'best_iter':clf.best_iteration_,
                                                'seed':seed,'max_max_depth':max_max_depth,'avg_max_depth':avg_max_depth}])],
                                                ignore_index=True)
    except Exception as e:
        print(f"Error with parameters: {param}")
        print(f"Error message: {e}")   

    

# save tje results in a file
results_df.to_csv(PATH_DATA_DTS+dts_name+"_LGBM_RESULTS9.csv", sep=",")
# save the model

# joblib.dump(clf, PATH_DATA_DTS+dts_name+"_LGBM_MODEL_R2.pkl")


combinations.__len__()=640
{'learning_rate': 0.001, 'max_depth': 10, 'num_leaves': 7, 'min_data_in_leaf': 0, 'feature_fraction': 0.05, 'lambda_l1': 0, 'lambda_l2': 0}
{'learning_rate': 0.001, 'max_depth': 10, 'num_leaves': 7, 'min_data_in_leaf': 0, 'feature_fraction': 0.05, 'lambda_l1': 0, 'lambda_l2': 1}
seed=5282
Try : {'learning_rate': 0.001, 'max_depth': 10, 'num_leaves': 7, 'min_data_in_leaf': 0, 'feature_fraction': 0.05, 'lambda_l1': 0, 'lambda_l2': 0}
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[14]	valid_0's custom_penalty: 3.59515
[[3418 2441 4687 1197 3007]
 [2433 3110 6094 1174 1939]
 [2221 2961 6653 1185 1730]
 [2648 3054 5929 1206 1913]
 [3617 1874 3703 1004 4552]]
Try : {'learning_rate': 0.001, 'max_depth': 10, 'num_leaves': 7, 'min_data_in_leaf': 0, 'feature_fraction': 0.05, 'lambda_l1': 0, 'lambda_l2': 1}


C:\Users\Benoit\AppData\Local\Temp\ipykernel_1444\2626994291.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df,


Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[14]	valid_0's custom_penalty: 3.60343
[[3451 2457 4679 1189 2974]
 [2450 3127 6055 1196 1922]
 [2239 2982 6621 1194 1714]
 [2689 3081 5901 1211 1868]
 [3673 1889 3665  997 4526]]
Try : {'learning_rate': 0.001, 'max_depth': 10, 'num_leaves': 7, 'min_data_in_leaf': 0, 'feature_fraction': 0.05, 'lambda_l1': 0, 'lambda_l2': 3}
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[14]	valid_0's custom_penalty: 3.61431
[[3473 2462 4649 1173 2993]
 [2468 3144 6036 1180 1922]
 [2264 2984 6596 1175 1731]
 [2717 3092 5874 1191 1876]
 [3701 1892 3653  964 4540]]
Try : {'learning_rate': 0.001, 'max_depth': 10, 'num_leaves': 7, 'min_data_in_leaf': 0, 'feature_fraction': 0.05, 'lambda_l1': 0, 'lambda_l2': 5}
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[14]	valid_0's custom_penalty: 3.61916
[[3509 2458 4635 1170

In [39]:

#test seed 

params = {
    'learning_rate': [0.001],  # 0.005
    'max_depth': [10],  # np.linspace(11,15,3, endpoint=True,dtype=int),
    'num_leaves': [7],  # 1023,1536],#np.linspace(15, 60,10, endpoint=True,dtype=int),
    'min_data_in_leaf': [25],
    'feature_fraction': [ 0.05],
    'lambda_l1': [0],#np.exp(np.random.default_rng().uniform(np.log(1e-4), np.log(10), 20)),
    'lambda_l2': [0],#np.exp(np.random.default_rng().uniform(np.log(1e-4), np.log(10), 20)),
}

# Generate all combinations of parameters
keys, values = zip(*params.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# Print the list of parameter combinations
print(f"{combinations.__len__()=}")
for combo in combinations[:2]:  # Print only the first 3 combinations for brevity
    print(combo)

# params=[
#     {'num_leaves': 511, 'min_data_in_leaf': 100, 'max_depth': 11, 'learning_rate': 0.01, 'lambda_l2': 0.025, 'lambda_l1': 0.01, 'feature_fraction': 0.4},
    # {'num_leaves': 1023, 'min_data_in_leaf': 100, 'max_depth': 15, 'learning_rate': 0.01, 'lambda_l2': 0.5, 'lambda_l1': 0.25, 'feature_fraction': 0.6}
#        ]

seeds=[np.random.default_rng().integers(0,10000) for i in range(10)]
print(f"{seed=}")
# loop over the parameters
for param in combinations:
    try:
        
        for seed in seeds:
            print(f"Try : {param} {seed=}")
            clf=lgb.LGBMClassifier(objective='multiclass',metric='custom',device_type=device_type,verbosity=-1,
                                seed=seed, num_class=5,n_estimators=1000,#np.random.default_rng().integers(0,10000),
                                **param)
            # train the model with the best parameters and eval set without log
            clf.fit(df_x_train_r, col_y_train, eval_set=[(df_x_val_r, col_y_val)],eval_metric=custom_penalty_score,
                    callbacks=[lgb.early_stopping(stopping_rounds=20)])#print_eval_metric

            # print(f"Best iteration: {clf.best_iteration_}")

            y_pred = clf.predict(df_x_train_r)
            accuracy_train = accuracy_score(col_y_train, y_pred)
            penalty_score=custom_penalty_score(col_y_train, y_pred)
            # print(f'Accuracy Train: {accuracy_train} Penalty Train: {penalty_score}')

            y_pred = clf.predict(df_x_val_r)
            accuracy_val = accuracy_score(col_y_val, y_pred)
            penalty_score=custom_penalty_score(col_y_val, y_pred)
            # print(f'Accuracy Val: {accuracy_val} Penalty Val: {penalty_score}')
            
            # Extract precision values
            precision_vals = precision_score(col_y_val, y_pred, average=None)
            precision_str ='precision:'+'-'.join([f'{i}:{precision_vals[i]:.2f}' for i in range(len(precision_vals))])
            
            # Extract recall values
            recall_vals = recall_score(col_y_val, y_pred, average=None)
            recall_str = 'recall:'+'-'.join([f'{i}:{recall_vals[i]:.2f}' for i in range(len(recall_vals))])

            # print(classification_report(col_y_val, y_pred))
            # print the confusion matrix
            print(metrics.confusion_matrix(col_y_val, y_pred))

            # Calculate the depth of each tree
            booster = clf.booster_
            tree_info = booster.dump_model()["tree_info"]
            # print(tree_info)
            max_depths = [calculate_tree_depth(tree["tree_structure"]) for tree in tree_info]
            avg_max_depth = sum(max_depths) / len(max_depths)
            max_max_depth = max(max_depths)
            # print(f"Max depth of trees: {avg_max_depth=} {max_max_depth=}")


            # save the values of param and accuracy_val in a dataframe
            results_df = pd.concat([results_df, 
                                    pd.DataFrame([{'params': param,'accuracy_train':round(accuracy_train,3),
                                                    'accuracy_val': round(accuracy_val,3),'penalty_score':round(penalty_score[1],3),
                                                    'precision':precision_str,'recall':recall_str,'best_iter':clf.best_iteration_,
                                                    'seed':seed,'max_max_depth':max_max_depth,'avg_max_depth':avg_max_depth}])],
                                                    ignore_index=True)
    except Exception as e:
        print(f"Error with parameters: {param}")
        print(f"Error message: {e}")   

    

# save tje results in a file
results_df.to_csv(PATH_DATA_DTS+dts_name+"_LGBM_RESULTS_seed.csv", sep=",")
# save the model

# joblib.dump(clf, PATH_DATA_DTS+dts_name+"_LGBM_MODEL_R2.pkl")


combinations.__len__()=1
{'learning_rate': 0.001, 'max_depth': 10, 'num_leaves': 7, 'min_data_in_leaf': 25, 'feature_fraction': 0.05, 'lambda_l1': 0, 'lambda_l2': 0}
seed=5282
Try : {'learning_rate': 0.001, 'max_depth': 10, 'num_leaves': 7, 'min_data_in_leaf': 25, 'feature_fraction': 0.05, 'lambda_l1': 0, 'lambda_l2': 0} seed=8758
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[6]	valid_0's custom_penalty: 3.80839
[[2894 3565 2862 2376 3053]
 [1947 4344 3798 2619 2042]
 [1911 4605 4008 2534 1692]
 [2159 4270 3722 2775 1824]
 [3862 2709 2259 2208 3712]]
Try : {'learning_rate': 0.001, 'max_depth': 10, 'num_leaves': 7, 'min_data_in_leaf': 25, 'feature_fraction': 0.05, 'lambda_l1': 0, 'lambda_l2': 0} seed=5370
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's custom_penalty: 3.03905
[[1888 3346 5742 3040  734]
 [1176 3374 7202 2459  539]
 [1281 3290 7149 2511  519]
 [1400 3280 6580 

In [33]:
results_df = pd.concat([results_df, pd.DataFrame([{'params': param, 'accuracy_val': accuracy_val}])], ignore_index=True)


C:\Users\Benoit\AppData\Local\Temp\ipykernel_5656\643497830.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{'params': param, 'accuracy_val': accuracy_val}])], ignore_index=True)


In [20]:
# print 10 random number from the expresion Real(1e-9,1000,'uniform')
# import Real
import numpy as np

log_min = np.log(1e-9)
log_max = np.log(1000)
log_uniform_random_numbers = np.exp(np.random.uniform(log_min, log_max, 10))

print(log_uniform_random_numbers)

[3.09096914e-02 4.63005868e+02 1.00491396e-09 6.74639811e+00
 1.41526469e-07 3.69786947e-04 2.34982012e-06 9.13282043e-05
 2.49505677e-07 3.54156661e+01]


In [ ]:
#  print the feature importance
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,df_x_train_r.columns)), columns=['Value','Feature'])
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()


In [ ]:
# print la map of the model
lgb.plot_importance(clf,importance_type='split',max_num_features=10,figsize=(20,10),title='Feature importance(split)')
lgb.plot_importance(clf,importance_type='gain',max_num_features=10,figsize=(20,10),title='Feature importance(gain)')
plt.show()


In [ ]:
y_pred = clf.predict(df_x_train_r)
accuracy = accuracy_score(col_y_train, y_pred)
print(f'Accuracy Train: {accuracy}')


y_pred = clf.predict(df_x_val_r)
accuracy = accuracy_score(col_y_val, y_pred)
print(f'Accuracy Val: {accuracy}')
print(classification_report(col_y_val, y_pred))
# print the confusion matrix
print(metrics.confusion_matrix(col_y_val, y_pred))

In [ ]:

lgb.plot_importance(clf,max_num_features=40, figsize=(7,6), title="LightGBM Feature Importance")
plt.show()
# save features importance in a file
feature_imp=pd.DataFrame()
feature_imp['features']=df_x_train_r.columns
feature_imp['importance']=clf.feature_importances_
feature_imp.to_csv(PATH_DATA_DTS+dts_name+"_LGBM_FEATURE_IMPORTANCE.csv", sep=",")


In [8]:
# load the model PARIS_TREND_1D_20D_V3_LGBM_MODEL_R2.pkl and print the characteristics
clf = joblib.load(PATH_DATA_DTS+"PARIS_TREND_1D_20D_V3_LGBM_MODEL_R2.pkl")
print(clf)


LGBMClassifier(device_type='gpu', feature_fraction=0.8, lambda_l1=0.2,
               lambda_l2=0.2, learning_rate=0.01, max_depth=11,
               metric='multi_logloss', min_data_in_leaf=100, n_estimators=1000,
               num_class=5, num_leaves=511, objective='multiclass', seed=42,
               verbosity=-1)
